In [30]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
BASE_URL="https://books.toscrape.com/catalogue/page-{}.html"
books=[]
page=1
while True:
    url=BASE_URL.format(page)
    response=requests.get(url)
    if response.status_code!=200:
        break
    soup=BeautifulSoup(response.text,"html.parser")
    articles=soup.find_all("article", class_="product_pod")
    if not articles:
        break
    for article in articles:
        title=article.h3.a["title"]
        price=article.find("p",class_="price_color").text.strip()
        availability=article.find("p",class_="instock availability").text.strip()
        star_rating=article.p.get("class")[1]
        books.append({
            "Title":title,
            "Price":price,
            "Availability":availability,
            "Star Rating":star_rating
        })
    page+=1
df_books=pd.DataFrame(books)
df_books.to_csv("books.csv", index=False)
URL="https://www.imdb.com/chart/top/"
response=requests.get(URL)
soup=BeautifulSoup(response.text, "html.parser")
movies=[]
rows=soup.select("tbody.lister-list tr")
for row in rows:
    rank=int(row.find("td", class_="titleColumn").get_text(strip=True).split(".")[0])
    title=row.find("td", class_="titleColumn").a.text
    year=row.find("span", class_="secondaryInfo").text.strip("()")
    rating=row.find("td", class_="ratingColumn imdbRating").strong.text
    movies.append({
        "Rank":rank,
        "Title":title,
        "Year":year,
        "IMDB Rating":rating
    })
df_movies=pd.DataFrame(movies)
df_movies.to_csv("imdb_top250.csv",index=False)
URL="https://www.timeanddate.com/weather/"
response=requests.get(URL)
soup=BeautifulSoup(response.text, "html.parser")
cities_data=[]
table=soup.find("table", class_="zebra tb-wt fw va-m tb-hover")
if table:
    rows=table.find("tbody").find_all("tr")
    for row in rows:
        cols=row.find_all("td")
        if len(cols)>=3:
            city=cols[0].get_text(strip=True)
            temp=cols[1].get_text(strip=True)
            condition=cols[2].get_text(strip=True)
            cities_data.append({
                "City":city,
                "Temperature":temp,
                "Condition":condition
            })
df_weather=pd.DataFrame(cities_data)
df_weather.to_csv("weather.csv",index=False)